In [3]:
#!/usr/bin/env python
%matplotlib inline  

#ipython magic to make ampmodule autoreload
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
#import scipy.integrate as spi
import pandas as pd
import scipy.stats as st 
#import sklearn.linear_model as slm 

#pull in the classes to do the sims!
import AMPmodule2

import resource
resource.setrlimit(resource.RLIMIT_NOFILE, (1000,-1)) #allow many plots

#all participants
VD=np.array(pd.read_csv('data/viral_dynamics.csv',usecols=range(1,8)))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [160]:
#simulate some viral loads and sample from AMP scheme

obstimes=np.array([0,2,4,8,12,24]) #observation weeks post first positive

nps=1000 #number of simulations, some will burn out
vlobs=np.zeros([nps,len(obstimes)])
rps=[]
ets=[]
fp=[]
ppt=0
while ppt<nps:
    
    exposure_day=np.random.uniform(0,28) #random exposure time in a 4 week interval 
    
    rp=np.random.randint(0,len(VD)) #choose a random individual VD set
    
    p=AMPmodule2.participant(name=ppt, et=exposure_day, tF=300, stop2=False, A0=1, 
                  vdp=VD[rp,:], pkp=[1e-5,0,0,0], pdp=[1e5,1], v_flg=False)

    t,sol=p.vd_sim()

    VL=np.log10(sol[:,6]/p.vol*1e3+1e-3)

    if (VL>-3).any():
        #find when VL>100 (detection limit)
        first_day_positive=t[np.argmax(VL>2)]

        #find first detected
        first_wk_detected=np.ceil(t[np.argmax(VL>2)]/7/4)*4

        oti=0
        for ot in obstimes:
            vlobs[ppt,oti]=VL[np.argmax(t>((ot+first_wk_detected)*7))]
            oti+=1

        fp.append(first_day_positive)
        rps.append(rp)
        ets.append(exposure_day-first_wk_detected*7) #relative to first detection

        ppt+=1
        
    #print(ppt)
    
#save dataframes
pd.DataFrame.to_csv(pd.DataFrame(np.array(vlobs),columns=obstimes),'sparsedata.csv')
pd.DataFrame.to_csv(pd.DataFrame(ets),'truetimes.csv')

In [ ]:
plt.boxplot(ets)

In [ ]:
plt.figure(figsize=(10,6))
for ppt in range(20):            
    plt.subplot(4,5,1+ppt)
    plt.plot(obstimes,vlobs[ppt,:],'-o')
    plt.ylim([-3,8])
plt.tight_layout()